In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import json

# Load the tokenizer and model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", token='')
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", token='')

# Set the device (use GPU if available)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Generate a response for a given question using the model.
  Args:
    question (str): The question to ask.
    context (str): Contextual information for the question.
    instructions (str): Additional instructions for the response.
    tokenizer: Tokenizer for input preparation.
    model: Pre-trained model for response generation.

  Returns:
    str: Generated answer.

In [ ]:
def ask_question(question, context, instructions, tokenizer, model):
    # Format the input prompt
    prompt = f"Context: {context}\n\nInstructions: {instructions}\n\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate a response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Answer:")[-1].strip()

Generate responses for a list of queries.
  Args:
    json_data (list): List of dictionaries containing query information.

  Returns:
    list: List of responses with query and corresponding answers.

In [ ]:
def generate_responses(json_data):
  responses = []
  tokenizer.pad_token = tokenizer.eos_token

  for item in json_data:
    # Call the ask_question function to get a response
    answer = ask_question(
      item['Query'],
      item['Context'],
      item.get('Additional Instructions', ''),
      tokenizer,
      model
    )
    responses.append({
      "query": item['Query'],
      "response": answer
    })
  return responses

# Load JSON data

In [ ]:
json_file = "/formula_calculation.json"
df = pd.read_json(json_file)
json_data = df.to_dict(orient='records')

# Test a single item from the JSON data

In [ ]:
first_item = json_data[0]
answer = ask_question(
  first_item['Query'],
  first_item['Context'],
  first_item.get('Additional Instructions', ''),
  tokenizer,
  model
)
print("Query:", first_item['Query'])
print("Additional Instructions:", first_item.get('Additional Instructions', ''))
print("Answer:", answer)

# Generate responses for all queries

In [ ]:
output_data = generate_responses(json_data)

# Save responses to a JSON file

In [ ]:
output_file = "generated_responses.json"
with open(output_file, 'w') as f:
  json.dump(output_data, f, indent=2)

print(f"Responses have been saved to '{output_file}'")